In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!pip install wget

In [2]:
#!pip install torch==1.7.0
!pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import KFold
import wget
import numpy as np

In [4]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
wget.download('https://raw.githubusercontent.com/donghyun-daniel/4_party_project/master/Review_crawler/csv_output/Preprocessing_review_pos_neg.csv')

100% [..........................................................................] 9183137 / 9183137

'Preprocessing_review_pos_neg (1).csv'

In [5]:
device = torch.device("cuda")

In [6]:
class NSMCDataset(Dataset):
    def __init__(self, csv_file):
        # 일부 값중에 NaN이 있음...
        self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
        # 중복제거
        self.dataset.drop_duplicates(subset=['document'], inplace=True)
        self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:3].values
        text = row[0]
        y = row[1]

        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, y


    
def data_concat(naverTrainData, naverTestData, hotelData): 
    #data_concat("ratings_train.txt", "ratings_test.txt", "Preprocessing_review_pos_neg.csv")
    trainRate = 0.5
    
    data1 = pd.read_table(naverTrainData)
    data2 = pd.read_table(naverTestData)
    data3 = pd.read_csv(hotelData)
    
    data3 = data3.drop(['len_text'], axis=1)
    data3['id'] = 1
    data3 = data3[['id', 'Text', 'Label']]
    data3.columns = ['id','document','label']
    data3 = data3.iloc[np.random.permutation(data3.index)].reset_index(drop=True)
    
    data3Train = data3.sample(frac = trainRate, random_state=2000)
    data3Test = data3.drop(data3Train.index)
    
    trainData = pd.concat([data1, data3Train], ignore_index=True)
    testData = pd.concat([data2, data3Test], ignore_index=True)
    
    trainData.to_csv('sampleTrain.txt', sep = '\t', index = False)
    testData.to_csv('sampleTest.txt', sep = '\t', index = False)

In [7]:
data_concat("ratings_train.txt", "ratings_test.txt", "Preprocessing_review_pos_neg.csv")

In [8]:
train_dataset = NSMCDataset("sampleTrain.txt")
test_dataset = NSMCDataset("sampleTest.txt")

                 id          label
count  1.965100e+05  196510.000000
mean   5.042975e+06       0.500010
std    3.885233e+06       0.500001
min    1.000000e+00       0.000000
25%    1.000000e+00       0.000000
50%    5.835446e+06       1.000000
75%    8.768834e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  9.948300e+04  99483.000000
mean   3.336797e+06      0.505051
std    3.957502e+06      0.499977
min    1.000000e+00      0.000000
25%    1.000000e+00      0.000000
50%    1.000000e+00      1.000000
75%    7.527450e+06      1.000000
max    1.027809e+07      1.000000


In [9]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# 에러나도 그냥 넘어가면 됨
model.load_state_dict(torch.load("model.pt"))

In [ ]:
model

In [ ]:
# GPU 메모리 터지는거 방지
gc.collect()
torch.cuda.empty_cache()

In [ ]:
epochs = 3
batch_size = 64

optimizer = AdamW(model.parameters(), lr=1e-5)

# 10 fold cross val
kfold = KFold(n_splits=10, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

kfold_losses = []
kfold_acc = []


for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
    print('----------------------------------')
    print(fold)

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_subsampler)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_subsampler)
    
    losses = []
    accuracies = []
    
    for i in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0
        batches = 0

        model.train()

        for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
            loss = F.cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)

            batches += 1
            if batches % 100 == 0:
                print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

        losses.append(total_loss)
        accuracies.append(correct.float() / total)
        print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)
    loss_avg = sum(losses) / len(losses)
    acc_avg = sum(accuracies) / len(accuracies)
    kfold_losses.append(loss_avg)
    kfold_acc.append(acc_avg)
    
    print('----------------------------------')

In [ ]:
kfold_losses, kfold_acc

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

In [ ]:
torch.save(model.state_dict(), "model.pt")

## 여기까지

In [ ]:
y_pred

In [ ]:
_, pre=torch.max(y_pred, 1)

In [ ]:
pre

In [ ]:
train_dataset

In [ ]:
train_loader

In [ ]:
input_ids_batch

In [ ]:
attention_masks_batch

In [ ]:
y_batch

In [ ]:
for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    break

In [ ]:
input_ids_batch

In [ ]:
attention_masks_batch

In [ ]:
y_batch

# 올릴 방법

- 어근 치환 ***
- k-fold crossval 

- pretrain 모델 변경 ***

- fine tuning
// - 데이터 추가 영화, 호텔, 쇼핑 데이터




In [ ]:
data = [1,2,3,4,5,6,7,8,9,10]